In [2]:
%run lta_datamall_data.ipynb

In [4]:
%run nearest_mrt_station_to_bus_stops.ipynb

25548it [00:06, 4033.48it/s]


In [6]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,"POLYGON ((11549880.181 155286.262, 11549893.09..."
1,0,None,None,MRT,HILLVIEW MRT STATION,"POLYGON ((11551320.421 151685.479, 11551324.86..."
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,"POLYGON ((11552265.127 149269.279, 11552255.26..."
3,0,None,None,MRT,HUME MRT STATION,"POLYGON ((11551478.151 150855.083, 11551485.53..."
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,"POLYGON ((11550666.118 153603.709, 11550698.62..."
...,...,...,...,...,...,...
225,0,None,None,MRT,KATONG PARK MRT STATION,"POLYGON ((11564432.509 144548.028, 11564447.75..."
226,0,None,None,MRT,MARINE TERRACE MRT STATION,"POLYGON ((11567908.523 145472.086, 11567908.42..."
227,0,None,None,MRT,TANJONG RHU MRT STATION,"POLYGON ((11563110.672 144334.178, 11563108.03..."
228,0,None,None,MRT,MARINE PARADE MRT STATION,"POLYGON ((11566600.443 144955.407, 11566582.69..."


# Brown Line

In [7]:
brown_line = ['WOODLANDS NORTH MRT STATION', 'WOODLANDS MRT STATION', 'WOODLANDS SOUTH MRT STATION',
              'SPRINGLEAF MRT STATION', 'LENTOR MRT STATION', 'MAYFLOWER MRT STATION',
              'BRIGHT HILL MRT STATION', 'UPPER THOMSON MRT STATION', 'CALDECOTT MRT STATION',
              'STEVENS MRT STATION', 'NAPIER MRT STATION', 'ORCHARD BOULEVARD MRT STATION',
              'ORCHARD MRT STATION', 'GREAT WORLD MRT STATION', 'HAVELOCK MRT STATION',
              'OUTRAM PARK MRT STATION', 'MAXWELL MRT STATION', 'SHENTON WAY MRT STATION',
              'MARINA BAY MRT STATION', 'GARDENS BY THE BAY MRT STATION', 'TANJONG RHU MRT STATION',
              'KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 'MARINE PARADE MRT STATION',
              'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION',
              'BEDOK SOUTH MRT STATION', 'SUNGEI BEDOK MRT STATION']

# filter brown line stations from mrt stations geodataframe
brown_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(brown_line)]

# order brown line stations based on list
brown_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(brown_line_stations_gdf['STN_NAM_DE'],
                                                       categories = brown_line,
                                                       ordered = True)
brown_line_stations_gdf = brown_line_stations_gdf.sort_values('STN_NAM_DE')
brown_line_stations_gdf


,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
192,0,None,None,MRT,WOODLANDS NORTH MRT STATION,"POLYGON ((11553290.504 161340.499, 11553308.78..."
196,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553655.571 159844.445, 11553714.07..."
177,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553305.733 159892.909, 11553302.44..."
191,0,None,None,MRT,WOODLANDS SOUTH MRT STATION,"POLYGON ((11554340.403 158819.168, 11554344.10..."
187,0,None,None,MRT,SPRINGLEAF MRT STATION,"POLYGON ((11556983.209 155700.488, 11556972.59..."
110,0,None,None,MRT,LENTOR MRT STATION,"POLYGON ((11559127.808 154037.405, 11559127.68..."
219,0,None,None,MRT,MAYFLOWER MRT STATION,"POLYGON ((11559099.098 152920.462, 11559094.58..."
186,0,None,None,MRT,BRIGHT HILL MRT STATION,"POLYGON ((11558731.827 151811.306, 11558686.82..."
217,0,None,None,MRT,UPPER THOMSON MRT STATION,"POLYGON ((11558480.214 150916.092, 11558483.67..."
218,0,None,None,MRT,CALDECOTT MRT STATION,"POLYGON ((11559462.068 148993.215, 11559463.61..."


In [8]:
# ordered list of brown line station coordinates
brown_line_stations_gdf['coordinates'] = brown_line_stations_gdf['geometry'].centroid
brown_line_stations_gdf

# create a linestring for the brown line route
from shapely.geometry import LineString
brown_line_route = LineString(brown_line_stations_gdf['coordinates'])

In [9]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

/var/folders/hy/5xrxtq_s4sl7k0tmmd7ty9mw0000gn/T/ipykernel_23872/1680479668.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


In [10]:
# calculate the overlapping distance of each bus route and the brown line
buffer_distance = 150
buffered_brown_line = brown_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_brown_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length
            })

overlap_distance = pd.DataFrame(overlap_distance_output)
overlap_distance

724it [00:00, 9154.80it/s]


,Bus Service,Direction,Overlap Distance
0,10,1,303.158745
1,10,2,304.681655
2,100,1,303.158745
3,100,2,305.338292
4,100A,1,0.000000
...,...,...,...
719,992,1,0.000000
720,992,2,0.000000
721,993,1,0.000000
722,9A,1,0.000000


In [11]:
# filter bus routes with overlap
bus_routes_overlap = overlap_distance[overlap_distance['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance
349,36,1,13497.671113
351,36B,1,11775.414400
392,47,1,6834.465678
225,196e,1,6707.001743
350,36A,1,6150.759588
...,...,...,...
184,178A,1,125.862879
183,178,2,124.004316
619,911A,1,104.939940
552,851e,1,46.125602


# Blue Line

In [12]:
blue_line = [
    'SUNGEI KADUT MRT STATION', 'BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 
    'SIXTH AVENUE MRT STATION', 'TAN KAH KEE MRT STATION', 'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 
    'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION', 'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 
    'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', 'BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 
    'GEYLANG BAHRU MRT STATION', 'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', 'BEDOK NORTH MRT STATION', 
    'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION', 'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'
]

# filter brown line stations from mrt stations geodataframe
blue_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(blue_line)]

# order brown line stations based on list
blue_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(blue_line_stations_gdf['STN_NAM_DE'],
                                                       categories = blue_line,
                                                       ordered = True)
blue_line_stations_gdf = blue_line_stations_gdf.sort_values('STN_NAM_DE')
blue_line_stations_gdf



,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,"POLYGON ((11550666.118 153603.709, 11550698.62..."
5,0,None,None,MRT,CASHEW MRT STATION,"POLYGON ((11551010.357 152545.896, 11551024.66..."
1,0,None,None,MRT,HILLVIEW MRT STATION,"POLYGON ((11551320.421 151685.479, 11551324.86..."
3,0,None,None,MRT,HUME MRT STATION,"POLYGON ((11551478.151 150855.083, 11551485.53..."
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,"POLYGON ((11552265.127 149269.279, 11552255.26..."
67,0,None,None,MRT,KING ALBERT PARK MRT STATION,"POLYGON ((11553177.948 148708.332, 11553217.08..."
33,0,None,None,MRT,SIXTH AVENUE MRT STATION,"POLYGON ((11554594.564 148204.756, 11554598.90..."
216,0,None,None,MRT,TAN KAH KEE MRT STATION,"POLYGON ((11555673.915 147692.277, 11555708.09..."
125,0,None,None,MRT,BOTANIC GARDENS MRT STATION,"POLYGON ((11556730.423 147258.342, 11556757.97..."
108,0,None,CC19_BTN STN.zip,MRT,BOTANIC GARDENS MRT STATION,"POLYGON ((11556592.183 147107.790, 11556577.02..."


In [13]:
# ordered list of blue line station coordinates
blue_line_stations_gdf['coordinates'] = blue_line_stations_gdf['geometry'].centroid
blue_line_stations_gdf

# create a linestring for the blue line route
from shapely.geometry import LineString
blue_line_route = LineString(blue_line_stations_gdf['coordinates'])

In [14]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

/var/folders/hy/5xrxtq_s4sl7k0tmmd7ty9mw0000gn/T/ipykernel_23872/1680479668.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


In [15]:
# calculate the overlapping distance of each bus route and the blue line
buffer_distance = 150
buffered_blue_line = blue_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_blue_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length
            })

overlap_distance_blue = pd.DataFrame(overlap_distance_output)
overlap_distance_blue

724it [00:00, 7803.90it/s]


,Bus Service,Direction,Overlap Distance
0,10,1,1085.722634
1,10,2,1091.514007
2,100,1,914.021723
3,100,2,914.707723
4,100A,1,304.650635
...,...,...,...
719,992,1,0.000000
720,992,2,0.000000
721,993,1,0.000000
722,9A,1,1183.825625


In [16]:
# filter bus routes with overlap with blue line
bus_routes_overlap_blue = overlap_distance_blue[overlap_distance_blue['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_blue

,Bus Service,Direction,Overlap Distance
478,67,2,16091.462313
477,67,1,14353.211924
164,170,1,13688.136207
252,23,1,12961.825736
165,170,2,12815.664331
...,...,...,...
53,123,2,191.817440
63,129A,1,30.953370
629,920,1,10.275598
630,922,1,10.275598


# Purple Line

In [17]:
purple_line = [
    'HARBOURFRONT MRT STATION',
    'OUTRAM PARK MRT STATION',
    'CHINATOWN MRT STATION',
    'CLARKE QUAY MRT STATION',
    'DHOBY GHAUT MRT STATION',
    'LITTLE INDIA MRT STATION',
    'FARRER PARK MRT STATION',
    'BOON KENG MRT STATION',
    'POTONG PASIR MRT STATION',
    'WOODLEIGH MRT STATION',
    'SERANGOON MRT STATION',
    'KOVAN MRT STATION',
    'HOUGANG MRT STATION',
    'BUANGKOK MRT STATION',
    'SENGKANG MRT STATION',
    'PUNGGOL MRT STATION'
]


# filter purple line stations from mrt stations geodataframe
purple_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(purple_line)]

# order purple line stations based on list
purple_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(purple_line_stations_gdf['STN_NAM_DE'],
                                                       categories = purple_line,
                                                       ordered = True)
purple_line_stations_gdf = purple_line_stations_gdf.sort_values('STN_NAM_DE')
purple_line_stations_gdf

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
157,0,None,CC29_HBF STN.zip,MRT,HARBOURFRONT MRT STATION,"POLYGON ((11557408.287 140823.192, 11557327.29..."
63,0,None,NE1_HBF STN.zip,MRT,HARBOURFRONT MRT STATION,"POLYGON ((11557256.560 140840.359, 11557256.27..."
201,0,None,None,MRT,OUTRAM PARK MRT STATION,"POLYGON ((11559308.372 142682.402, 11559298.43..."
117,0,None,NE3_OTP STN.zip,MRT,OUTRAM PARK MRT STATION,"POLYGON ((11559350.232 142623.057, 11559353.15..."
84,0,None,C109_S_OTP_MAIN_A_R19_P2.zip,MRT,OUTRAM PARK MRT STATION,"POLYGON ((11559237.753 142695.209, 11559251.83..."
169,0,None,None,MRT,CHINATOWN MRT STATION,"POLYGON ((11559751.172 143121.613, 11559752.03..."
92,0,None,NE4_CNT STN.zip,MRT,CHINATOWN MRT STATION,"POLYGON ((11559695.295 142864.198, 11559671.98..."
91,0,None,NE5_CQY STN.zip,MRT,CLARKE QUAY MRT STATION,"POLYGON ((11560054.558 143242.668, 11560050.58..."
6,0,None,None,MRT,DHOBY GHAUT MRT STATION,"POLYGON ((11559965.996 144666.945, 11560032.60..."
90,0,None,NE6_DBG STN.zip,MRT,DHOBY GHAUT MRT STATION,"POLYGON ((11560057.669 144610.631, 11560099.83..."


In [18]:
# ordered list of purple line station coordinates
purple_line_stations_gdf['coordinates'] = purple_line_stations_gdf['geometry'].centroid
purple_line_stations_gdf

# create a linestring for the purple line route
from shapely.geometry import LineString
purple_line_route = LineString(purple_line_stations_gdf['coordinates'])

In [19]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

/var/folders/hy/5xrxtq_s4sl7k0tmmd7ty9mw0000gn/T/ipykernel_23872/1680479668.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


In [20]:
# calculate the overlapping distance of each bus route and the purple line
buffer_distance = 150
buffered_purple_line = purple_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_purple_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length
            })

overlap_distance_purple = pd.DataFrame(overlap_distance_output)
overlap_distance_purple

724it [00:00, 7289.87it/s]


,Bus Service,Direction,Overlap Distance
0,10,1,595.777236
1,10,2,636.280118
2,100,1,1981.580285
3,100,2,1628.347311
4,100A,1,1385.803049
...,...,...,...
719,992,1,0.000000
720,992,2,0.000000
721,993,1,0.000000
722,9A,1,0.000000


In [21]:
# filter bus routes with overlap with purple line
bus_routes_overlap_purple = overlap_distance_purple[overlap_distance_purple['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_purple

,Bus Service,Direction,Overlap Distance
19,107M,1,11014.797338
108,147,2,7635.060619
5,101,1,6218.377800
539,82,1,6197.415788
18,107,2,5808.077146
...,...,...,...
310,3,1,100.239040
12,105,2,73.741523
22,109A,1,71.285963
8,102B,1,70.896183
